In [1]:
# Define F1 score

import numpy as np


def pairwise_operator(codes, method):
    """
    Pairwsise operator based on a method and a list of predictions (e.g., lists of offsets)
    >>> assert pairwise_operator([[],[],[]], f1) == 1
    :param codes: a list of lists of predicted offsets
    :param method: a method to use to compare all pairs
    :return: the mean score between all possible pairs (excl. duplicates)
    """
    pairs = []
    for i,coderi in enumerate(codes):
        for j,coderj in enumerate(codes):
            if j>i:
                pairs.append(method(coderi, coderj))
    return np.mean(pairs)


def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1 if len(predictions)==0 else 0
    nom = 2*len(set(predictions).intersection(set(gold)))
    denom = len(set(predictions))+len(set(gold))
    return nom/denom

In [22]:
# Read the train data

import pandas as pd
train = pd.read_csv("tsd_test.csv")
texts = list(train["text"])

In [4]:
train.head()

,spans,text
0,"[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...",That's right. They are not normal. And I am st...
1,"[81, 82, 83, 84, 85, 86]","""Watch people die from taking away their healt..."
2,[],tens years ago i contacted the PDR and suggest...
3,[],The parallels between the ANC and the Sicilian...
4,[],Intel Community: ‘How can we work for a Presid...


## Extract ground-truth spans

In [ ]:
# Make a list of list with character offsets of dataset

spans = []

for i, row in train.iterrows():
    to_append_list = [int(x) for x in str(train["spans"].iloc[i])[1:len(str(train["spans"].iloc[i]))-1].split(", ") if len(x) > 0]
    spans.append(to_append_list)

In [6]:
# Group non-contiguous spans

import more_itertools as mit
span_grouped = []

for i in range(len(spans)):
    span_grouped.append([list(group) for group in mit.consecutive_groups(spans[i])])

In [7]:
# Find starting and ending offset of non-contiguos offset


final = []

for i in range(len(span_grouped)):
    span_grouped_just_group_sentence = []
    for ranges in span_grouped[i]:
        span_grouped_just_group_groups = []
        #print(ranges)
        if len(ranges) > 0:
            span_grouped_just_group_groups = [ranges[0],ranges[-1]]
        
        span_grouped_just_group_sentence.append(span_grouped_just_group_groups)
        
    final.append(span_grouped_just_group_sentence)     

In [8]:
count_spans = [item for sublist in final for item in sublist]

In [10]:
# Extract ground-truth spans

only_spans = []

for i in range(len(final)):
    sentence_spans = []
    for j in range(len(final[i])):
        text = texts[i][final[i][j][0]:final[i][j][1]+1]
        sentence_spans.append(text)
    only_spans.append(sentence_spans)
        

In [18]:
flat_list = [item for sublist in only_spans for item in sublist]

In [13]:
all_length = []
count_span = 0
for sentence_span in only_spans:
    sentence_span_length = []
    for span in sentence_span:
        sentence_span_length.append(len(span.split(" ")))
    all_length.append(sentence_span_length)

## Extract predicted spans 

In [478]:
# Read predicted spans

with open("spans-pred-biaffine.txt", 'r') as file1:
    lines = file1.readlines()

all_answers = []

for line in lines:
    if not (line.isspace()):
        feats = eval(line.split('\t')[1])
        all_answers.append(feats)

In [479]:
# Make a list of list with character offsets of dataset

spans_predicted = []

for i in range(len(all_answers)):
    to_append_list = [int(x) for x in str(all_answers[i])[1:len(str(all_answers[i]))-1].split(", ") if len(x) > 0]
    spans_predicted.append(to_append_list)

In [480]:
# Group non-contiguous spans

import more_itertools as mit
span_grouped_predicted = []

for i in range(len(spans_predicted)):
    span_grouped_predicted.append([list(group) for group in mit.consecutive_groups(spans_predicted[i])])

In [481]:
# Find starting and ending offset of non-contiguos offset

final_predicted = []

for i in range(len(span_grouped_predicted)):
    span_grouped_just_group_sentence = []
    for ranges in span_grouped_predicted[i]:
        span_grouped_just_group_groups = []
        #print(ranges)
        if len(ranges) > 0:
            span_grouped_just_group_groups = [ranges[0],ranges[-1]]
        
        span_grouped_just_group_sentence.append(span_grouped_just_group_groups)
        
    final_predicted.append(span_grouped_just_group_sentence)    

In [483]:
# Extract predicted spans

only_spans_predicted = []

for i in range(len(final_predicted)):
    sentence_spans = []
    for j in range(len(final_predicted[i])):
        text = texts[i][final_predicted[i][j][0]:final_predicted[i][j][1]+1]
        sentence_spans.append(text)
    only_spans_predicted.append(sentence_spans)

In [485]:
# Length Calculation

all_length_predicted = []
count_span = 0
for sentence_span in only_spans_predicted:
    sentence_span_length = []
    for span in sentence_span:
        sentence_span_length.append(len(span.split(" ")))
    all_length_predicted.append(sentence_span_length)

## Custom Filters 

In [486]:
# Dictionary of top 50 occuring cuss words (single word)

tox_dict = {'Damn': 34,
 'Dumb': 20,
 'IDIOT': 16,
 'Idiot': 41,
 'Idiots': 38,
 'Pathetic': 27,
 'Ridiculous': 15,
 'STUPID': 32,
 'Stupid': 82,
 'arrogant': 19,
 'ass': 55,
 'asshole': 20,
 'bitch': 19,
 'buffoon': 24,
 'bullshit': 30,
 'clown': 40,
 'clowns': 29,
 'corrupt': 24,
 'coward': 30,
 'crap': 98,
 'crazy': 21,
 'damn': 70,
 'darn': 14,
 'disgusting': 32,
 'dumb': 151,
 'dumber': 18,
 'fool': 138,
 'foolish': 36,
 'fools': 87,
 'fucking': 16,
 'garbage': 34,
 'gay': 20,
 'hypocrisy': 20,
 'hypocrite': 66,
 'hypocrites': 31,
 'hypocritical': 18,
 'idiocy': 30,
 'idiot': 538,
 'idiotic': 101,
 'idiots': 340,
 'ignorance': 35,
 'ignorant': 196,
 'imbecile': 33,
 'incompetent': 15,
 'jerk': 45,
 'jerks': 17,
 'kill': 16,
 'liar': 47,
 'loser': 68,
 'losers': 50,
 'lunatic': 20,
 'moron': 165,
 'moronic': 19,
 'morons': 83,
 'nonsense': 14,
 'pathetic': 118,
 'pig': 17,
 'pussy': 24,
 'racist': 52,
 'ridiculous': 99,
 'scum': 43,
 'scumbag': 17,
 'shit': 53,
 'sick': 16,
 'silly': 45,
 'stupid': 1002,
 'stupidest': 17,
 'stupidity': 217,
 'suck': 15,
 'sucks': 21,
 'traitor': 16,
 'trash': 29,
 'troll': 31,
 'trolls': 17}

tox_dict_key = list(tox_dict.keys())

In [487]:
# Filter or Predicted (either length or toxic word)

for i in range(len(only_spans_predicted)):
    new_span_sentence = []
    new_char_offset_sentence = []
    for j in range(len(only_spans_predicted[i])):
        if (len(only_spans_predicted[i][j].split(" ")) >= 5):
 #           if only_spans_predicted[i][j] in tox_dict_key:
            new_span_sentence.append(only_spans_predicted[i][j])
            new_char_offset_sentence.append(final_predicted[i][j])
        
    only_spans_predicted[i] = new_span_sentence
    final_predicted[i] = new_char_offset_sentence


In [488]:
# Filter for GT (either length or toxic word)

for i in range(len(only_spans)):
    new_span_sentence = []
    new_char_offset_sentence = []
    for j in range(len(only_spans[i])):
        if  (len(only_spans[i][j].split(" ")) >= 5) :
#            if only_spans[i][j] in tox_dict_key:
            new_span_sentence.append(only_spans[i][j])
            new_char_offset_sentence.append(final[i][j])
        
    only_spans[i] = new_span_sentence
    final[i] = new_char_offset_sentence

In [492]:
# Calculate offsets for Gold (after filter)

for i in range(len(final)):
    final_answer = []
    #print(final[i])
    for j in range(len(final[i])):
        answer = [number for number in range(final[i][j][0],final[i][j][1] + 1)]
        final_answer.extend(answer)
    final[i] = final_answer

In [493]:
# Calculate offsets for Predicted (after filter)

for i in range(len(final_predicted)):
    final_answer = []
    #print(final[i])
    for j in range(len(final_predicted[i])):
        answer = [number for number in range(final_predicted[i][j][0],final_predicted[i][j][1] + 1)]
        final_answer.extend(answer)
    final_predicted[i] = final_answer

## F1 Ca

In [496]:
import numpy as np
f1s = []

for i in range(2000):
    if (len(final_predicted[i]) == 0) and (len(final[i]) == 0):
        continue
    else:
        f1s.append(f1(final_predicted[i],final[i]))

In [497]:
np.mean(f1s)

0.0